## Here, we are training a StyleGAN2 model from scratch to compare to StyleGAN2-ADA on the same dataset

In [20]:
from glob import glob
from tqdm import tqdm
from pathlib import Path
import sys
import dotenv
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

dotenv.load_dotenv(override=True)
WORK = os.getenv('WORK')
sys.path.insert(0, f'{WORK}/ADA_Project')
sys.path.insert(0, f'{WORK}/ADA_Project/stylegan2')
import DeepFAMS

WORK, PROJ_DIR = DeepFAMS.utils.set_env()

In [21]:
DATASET_NAME = 'metfaces'

In [22]:
RAW_IMGS_DIR, RESIZED_IMGS_DIR, DATA_CUSTOM_DIR, TRAIN_RUNS_DIR = DeepFAMS.utils.return_dirs(
    PROJ_DIR, DATASET_NAME)

In [23]:
RAW_IMGS_DIR_ = f'{PROJ_DIR}/datasets/metfaces-release/images'
RESIZED_IMGS_DIR_ = f'{PROJ_DIR}/datasets/metfaces_resized_imgs'
TRAIN_RUNS_DIR_ = f'{Path(TRAIN_RUNS_DIR).parent}/StyleGAN2_{Path(TRAIN_RUNS_DIR).name}'

In [24]:
raw_imgs = glob(f'{RAW_IMGS_DIR_}/*')
print(len(raw_imgs))

1336


In [25]:
# for x in tqdm(raw_imgs):
#     DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

In [26]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR_}/*"))}')

Raw: 1336, Resized: 1336


In [ ]:
# DeepFAMS.preprocessing.tf_record_exporter(
#     tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

In [ ]:
# Initial run to create the first checkpoint
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/6.1
module load cuda/10.0
conda activate stylegan2
python3 {PROJ_DIR}/stylegan2/run_training.py \
    --num-gpus=2 \
    --data-dir=$WORK/ADA_Project/datasets \
    --result-dir={TRAIN_RUNS_DIR_} \
    --config=config-f \
    --mirror-augment=true \
    --total-kimg=1 \
    --dataset={Path(DATA_CUSTOM_DIR).name}''', PROJ_DIR)

Local submit - run_dir: /work/chaselab/malyetama/ADA_Project/training_runs/StyleGAN2_metfaces_training-runs/00000-stylegan2-metfaces_custom-2gpu-config-f

dnnlib: Running training.training_loop.training_loop() on localhost...

Streaming data using training.dataset.TFRecordDataset...

Dataset shape = [3, 512, 512]

Dynamic range = [0, 255]

Label size    = 0

Constructing networks...

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.

Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.



G                             Params    OutputShape         WeightShape     

---                           ---       ---                 ---             

latents_in                    -         (?, 512)            -               

labels_in                     -         (?, 0)              -               

lod                           -         ()                  -               

dlatent_avg                   -         (512,)         

Building TensorFlow graph...

Initializing logs...

Training for 1 kimg...



tick 0     kimg 0.1      lod 0.00  minibatch 32   time 42s          sec/tick 42.4    sec/kimg 330.95  maintenance 0.0    gpumem 8.3



In [ ]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

In [ ]:
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/6.1
module load cuda/10.0
conda activate stylegan2
python3 {PROJ_DIR}/stylegan2/run_training.py \
    --num-gpus=2 \
    --data-dir={PROJ_DIR}/datasets \
    --result-dir={TRAIN_RUNS_DIR_} \
    --config=config-f \
    --mirror-augment=true \
    --dataset={Path(DATA_CUSTOM_DIR).name}
    --resume-pkl={latest_snap}''', PROJ_DIR)